### HuggingFace Interface

In [64]:
from allennlp.modules.elmo import Elmo, batch_to_ids
from sacremoses import MosesTokenizer
from transformers import (
    PretrainedConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
)


class ElmoConfig(PretrainedConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = 1024
        self.num_hidden_layers = 1
        self.is_encoder_decoder = False


class ElmoModel(PreTrainedModel):
    def __init__(self, options_file, weights_file):
        super().__init__(config=ElmoConfig())
        self.elmo_model = Elmo(options_file, weights_file, 1, dropout=0)

    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
    ):
        return self.elmo_model(input_ids)["elmo_representations"][0]

    @staticmethod
    def from_pretrained(path):
        options_file = (
            "/home/kyle/elk/elk/rnn/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
        )
        weights_file = (
            "/home/kyle/elk/elk/rnn/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
        )
        return ElmoModel(options_file, weights_file)


class ElmoTokenizer(PreTrainedTokenizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tokenizer = MosesTokenizer()

    def __call__(self, text, return_tensors, truncation):
        sequences = text if isinstance(text, list) else [text]
        tokens = [
            self.tokenizer.tokenize(sequence, escape=False) for sequence in sequences
        ]
        character_ids = batch_to_ids(tokens)  # type: ignore
        return character_ids

    @staticmethod
    def from_pretrained(path):
        return ElmoTokenizer()

sequences = ["Is this an ELMo representation? Yes", "Is this an ELMo representation? No", "Is this an ELMo representation? Maybe"]
elmo_tokenizer = ElmoTokenizer()
tokenized_sequences = elmo_tokenizer(sequences, return_tensors="pt", truncation=True)
elmo_model = ElmoModel.from_pretrained("elmo")
embeddings = elmo_model(tokenized_sequences)
print(embeddings.shape)
embeddings


torch.Size([3, 7, 1024])


tensor([[[-0.8412, -0.1165,  0.5267,  ...,  0.4512, -0.1392,  0.2720],
         [-0.6446, -0.0498,  0.3769,  ...,  0.4740, -0.1965,  0.0583],
         [-0.2392,  1.1553,  0.1504,  ..., -0.0441, -0.4352,  0.2593],
         ...,
         [-0.3401, -0.1522,  0.0504,  ...,  0.1691, -0.0028, -0.2495],
         [-0.1372,  0.0519,  0.0663,  ...,  1.1651,  0.6015, -0.1530],
         [-0.2489, -0.0674,  0.5087,  ...,  0.5525, -0.2275,  0.5592]],

        [[-0.8412, -0.1165,  0.5267,  ...,  0.5106, -0.0239,  0.3420],
         [-0.6446, -0.0498,  0.3769,  ...,  0.5673, -0.0058,  0.2063],
         [-0.2392,  1.1553,  0.1504,  ..., -0.0058, -0.2897,  0.3701],
         ...,
         [-0.3401, -0.1522,  0.0504,  ...,  0.1757, -0.0016,  0.1202],
         [-0.1372,  0.0519,  0.0663,  ...,  1.1888,  0.6638, -0.0508],
         [ 0.0425,  0.0044,  0.2477,  ...,  0.1274, -0.0031,  0.7462]],

        [[-0.8412, -0.1165,  0.5267,  ...,  0.3903, -0.1275,  0.1248],
         [-0.6446, -0.0498,  0.3769,  ...,  0

## Isolate LSTM

In [65]:
bilm_lstm = elmo_model.elmo_model._elmo_lstm
lstm_token_embedder = bilm_lstm._token_embedder
internal_embeddings = lstm_token_embedder(tokenized_sequences)
print(internal_embeddings["token_embedding"].shape)
internal_embeddings

torch.Size([3, 9, 512])


{'mask': tensor([[True, True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True, True, True]]),
 'token_embedding': tensor([[[-8.5709e+00, -9.9289e+00,  4.0575e+00,  ...,  9.1735e+00,
            8.3215e+00, -6.9233e+00],
          [-1.3800e-02, -2.2660e-01, -3.4851e-02,  ...,  1.9179e-01,
            4.6514e-02, -2.2795e-02],
          [-6.0722e-02,  5.1529e-02,  1.8452e-02,  ...,  2.3105e-01,
            3.6155e-03,  4.1534e-02],
          ...,
          [ 6.4416e-01, -7.7645e-02,  7.0715e-02,  ...,  6.5416e-01,
            5.1353e-01, -2.2917e-02],
          [-3.8933e-01,  8.6538e-01,  1.1729e-01,  ...,  4.5468e-01,
           -9.4481e-02,  2.5647e-01],
          [-1.2410e-01, -3.2147e+00, -6.1114e+00,  ...,  9.8041e+00,
            1.9380e+00, -8.1160e+00]],
 
         [[-8.5709e+00, -9.9289e+00,  4.0575e+00,  ...,  9.1735e+00,
            8.3215e+00, -6.9233e+00],
         

In [66]:
activations = bilm_lstm(tokenized_sequences)["activations"]
activations[0].shape

torch.Size([3, 9, 1024])

In [87]:
internal_bilm = bilm_lstm._elmo_lstm
bilm_hidden_states = internal_bilm(inputs=internal_embeddings["token_embedding"], mask=internal_embeddings["mask"])
print(bilm_hidden_states.shape)
print(internal_bilm.hidden_size)

# Calling the internal BiLM directly gives the hidden states for each layer. There are two layers.
# The output tensor is of shape (2, batch_size, sequence_length, 1012 (512 * 2?))
bilm_hidden_states

torch.Size([2, 3, 9, 1024])
512


tensor([[[[ 0.0179,  0.0713,  0.0110,  ...,  0.1264,  0.1231,  0.0258],
          [-0.4891,  0.1080,  0.3984,  ...,  0.3566, -0.3906,  0.2677],
          [-0.6844,  0.0765,  0.0339,  ...,  0.3921, -0.6338,  0.0184],
          ...,
          [-0.3669,  0.2947, -0.0701,  ...,  0.1141, -0.2551,  0.0222],
          [ 0.0987,  0.0519,  0.5976,  ...,  0.3036,  0.1679,  0.4183],
          [ 0.2429,  0.6859,  0.1255,  ...,  0.1900, -0.0090,  0.0093]],

         [[ 0.0178,  0.0720,  0.0108,  ...,  0.1288,  0.1253,  0.0255],
          [-0.4869,  0.1092,  0.3964,  ...,  0.3726, -0.3611,  0.2696],
          [-0.6809,  0.0774,  0.0320,  ...,  0.4158, -0.5869,  0.0081],
          ...,
          [-0.3650,  0.2953, -0.0698,  ...,  0.1185, -0.1778, -0.0985],
          [ 0.1864, -0.0251,  0.4565,  ...,  0.1383,  0.2780,  0.4876],
          [ 0.2692,  0.6085,  0.1256,  ...,  0.1899, -0.0089,  0.0096]],

         [[ 0.0178,  0.0718,  0.0108,  ...,  0.1197,  0.1168,  0.0254],
          [-0.4874,  0.1090,  

In [86]:
bilm_hidden_states[0][0][0]

tensor([0.0183, 0.0676, 0.0123,  ..., 0.1266, 0.1232, 0.0253])

In [76]:
# Output activations from the internal BiLM
print(activations[0].shape)
bilm_lstm(tokenized_sequences)["activations"]

torch.Size([3, 9, 1024])


[tensor([[[-8.5709e+00, -9.9289e+00,  4.0575e+00,  ...,  9.1735e+00,
            8.3215e+00, -6.9233e+00],
          [-1.3800e-02, -2.2660e-01, -3.4851e-02,  ...,  1.9179e-01,
            4.6514e-02, -2.2795e-02],
          [-6.0722e-02,  5.1529e-02,  1.8452e-02,  ...,  2.3105e-01,
            3.6155e-03,  4.1534e-02],
          ...,
          [ 6.4416e-01, -7.7645e-02,  7.0715e-02,  ...,  6.5416e-01,
            5.1353e-01, -2.2917e-02],
          [-3.8933e-01,  8.6538e-01,  1.1729e-01,  ...,  4.5468e-01,
           -9.4481e-02,  2.5647e-01],
          [-1.2410e-01, -3.2147e+00, -6.1114e+00,  ...,  9.8041e+00,
            1.9380e+00, -8.1160e+00]],
 
         [[-8.5709e+00, -9.9289e+00,  4.0575e+00,  ...,  9.1735e+00,
            8.3215e+00, -6.9233e+00],
          [-1.3800e-02, -2.2660e-01, -3.4851e-02,  ...,  1.9179e-01,
            4.6514e-02, -2.2795e-02],
          [-6.0722e-02,  5.1529e-02,  1.8452e-02,  ...,  2.3105e-01,
            3.6155e-03,  4.1534e-02],
          ...,
    

## biLM HuggingFace Port

In [90]:
class ElmoBiLMTokenizer(PreTrainedTokenizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.elmo_tokenizer = ElmoTokenizer()
        self.token_embedder = ElmoModel.from_pretrained("elmo").elmo_model._elmo_lstm._token_embedder

    def __call__(self, text, return_tensors, truncation):
        tokenized_text = self.elmo_tokenizer(text, return_tensors, truncation)
        embeddings = self.token_embedder(tokenized_text)
        return embeddings
        
    @staticmethod
    def from_pretrained(path):
        return ElmoBiLMTokenizer()


class ElmoBiLMConfig(PretrainedConfig):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = 1024
        self.num_hidden_layers = 2
        self.is_encoder_decoder = False

    
class ElmoBiLM(PreTrainedModel):
    def __init__(self):
        super().__init__(config=ElmoBiLMConfig())
        self.elmo_lstm = ElmoModel.from_pretrained("elmo").elmo_model._elmo_lstm._elmo_lstm
        
    def forward(
        self,
        input_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
    ):
        inputs = input_ids
        return self.elmo_lstm(inputs=input_ids, mask=attention_mask)

bilm_tokenizer = ElmoBiLMTokenizer()
tokenizer_batch = bilm_tokenizer(sequences, return_tensors="pt", truncation=True)
bilm_model = ElmoBiLM()
hidden_states = bilm_model(tokenizer_batch["token_embedding"], attention_mask=tokenizer_batch["mask"])
hidden_states

tensor([[[[ 2.7004e-02, -5.8437e-03,  3.8740e-02,  ...,  1.1279e-01,
            8.2891e-02,  3.0009e-02],
          [-9.0796e-01, -1.6039e-01,  8.3736e-01,  ...,  4.3962e-01,
           -1.0700e-01,  2.9674e-01],
          [-6.9313e-01,  1.0666e-02,  3.2420e-01,  ...,  3.0632e-01,
           -3.9687e-01, -1.9246e-02],
          ...,
          [-4.8125e-01,  1.9718e-01,  3.3368e-03,  ...,  2.3145e-01,
           -3.2376e-02, -2.3980e-01],
          [ 1.3262e-01, -1.0354e-01,  7.3106e-01,  ...,  6.5551e-01,
           -3.6035e-01,  1.1051e+00],
          [ 2.4790e-01,  5.9119e-01,  1.9945e-01,  ...,  1.6856e-01,
           -3.2241e-02,  4.7876e-02]],

         [[ 2.7004e-02, -5.8437e-03,  3.8740e-02,  ...,  1.1832e-01,
            9.5615e-02,  2.6538e-02],
          [-9.0796e-01, -1.6039e-01,  8.3736e-01,  ...,  4.0804e-01,
            7.6256e-03,  3.4817e-01],
          [-6.9313e-01,  1.0666e-02,  3.2420e-01,  ...,  2.6200e-01,
           -2.6565e-01,  9.1764e-02],
          ...,
     

In [91]:
hidden_states.shape

torch.Size([2, 3, 9, 1024])